<a href="https://colab.research.google.com/github/Minsung-commit/TIL/blob/master/Instagram_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !sudo apt-get install python3-dev; pip3 install konlpy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
python3-dev is already the newest version (3.6.7-1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [ ]:
# !sudo apt-get install g++ openjdk-8-jdk

Reading package lists... Done
Building dependency tree       
Reading state information... Done
g++ is already the newest version (4:7.4.0-1ubuntu2.3).
openjdk-8-jdk is already the newest version (8u292-b10-0ubuntu1~18.04).
0 upgraded, 0 newly installed, 0 to remove and 37 not upgraded.


In [36]:
!pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 5.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 15.7 MB 180 kB/s 
     |████████████████████████████████| 11.3 MB 38.4 MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136897 sha256=3fa05c88ae2be9e6d7c873b4c3677c39ec3dea230cc480068595909c95c00613
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis
  Attempting uninstall: numpy
    Found existing installation: numpy 1.19.5
    Uninstalling numpy-1.19.5:
      Successfully uninstalled numpy-1.19.5
  Attempting uninstall: pandas
    Found existing installation: pandas 1.1.5
    Uninstalling pandas-1.1.5:
      Successfully uninstalled pandas-1.1.5
ERROR: pip's dependency resolver does not currently take int

In [ ]:
import pandas as pd
import numpy as np
import re
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

In [ ]:
# from konlpy.tag import Hannanum
# from konlpy.tag import Okt 

In [2]:
data = pd.read_csv('./tot_dataset.csv') #데이터셋 로드

In [ ]:
data.info() #데이터셋 확인

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 609 entries, 0 to 608
Data columns (total 8 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  609 non-null    int64 
 1   content     609 non-null    object
 2   date        609 non-null    object
 3   like        609 non-null    int64 
 4   tags        609 non-null    object
 5   name        609 non-null    object
 6   overlap     609 non-null    int64 
 7   place       248 non-null    object
dtypes: int64(3), object(5)
memory usage: 38.2+ KB


In [ ]:
data.head()

In [ ]:
# data.nlargest(20, 'overlap') #중복콘텐츠 확인

In [ ]:
# data.nlargest(20, 'like') #like 높은 숙소 확인

In [ ]:
data.overlap.value_counts() # 중복값 counts 확인

1    363
2    146
3     60
4     30
5      6
6      3
8      1
Name: overlap, dtype: int64

#토큰화

In [3]:
!pip install tomotopy kiwipiepy pyvis

In [4]:
import tomotopy as tp # 토픽 모델링에 사용할 패키지
from kiwipiepy import Kiwi # 한국어 형태소 분석에 사용할 패키지
from pyvis.network import Network # 네트워크 시각화에 사용할 패키지

In [5]:
data = data[['name','content']] #데이터셋 재구성
text = data.content

한국어 전처리

In [6]:
stop_words = '의 가 이 은 들 는 좀 잘 속초 걍 과 했 거 해서 게 찍 느낌 많이 듯 뷰 박 링크 인스타그램 할인 성수기 홈페이지 주말 블로그 인원 추가 ㅎㅎ 너무 게스트 넘 하우스 드리다 이용 위치 쓰다 진짜 넘 찍 거 먹 ㅠㅠ ㅎㅎㅎ ㅠ 물 였 ㅠㅠㅠ ㅠㅠ ㅠ ㅋㅋㅋ ㅋㅋ ㅋ ㅎㅎㅎ ㅎㅎ ㅎ 화장실 도 뭐 오픈 최대 준비 룸 빵 거 많이 방법 달리 스럽다 특별자치도 를 에어비 으로 자 에 와 한 하다 아 휴 아이구 포스팅 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 이제 분 도움 여 ㅁ ㅎ ㄶ 얼마간 둥 오랜만 약간 체크 체크아웃 가격 정보 비 수기 평일 기준 약 전국  예약 되어다 스마트 빔 블루투스 스피커 이 외 드라이어 다리미 구비 되어다 있다 주방 음식 조리 가능하다 환기 제한 전남 제부도 있다 냄새 나 요리 삼가다 경주시 부탁드리다 후 번길 문의 다 가격 수기 비성수기 주소  좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 포항 우도 양양 전주시 통영 제천 여수시 순천 고성 합천 한림읍 전주 경북 구좌읍 돌산읍 태안 하동 포항시 제주시 밀양 양평 울산 무무 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 춘천 층 남해 스테이 서울 여수 거제 추천 경주 곳 객실 이다 강원도 강원 홍천 부산 영도 호 객실 교동 풀빌라 빌라 풀 스튜디오 펜션 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 감성 숙소 호텔 제주 제주도 강원도 강릉 속초 속초시 에어비앤비 명 생각 시간 그녀 수 약 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓 사실 이렇 점 싶 말 정도 좀 원 잘 통하 ㅆ 재 채 독 롭 도란도 팅 감성비앤비 공간 참여 이벤트 당첨자 기간 부산 #부산숙소 #부산에어비앤비 #부산여행 경기 경주시 있는 한 #경주숙소 강원도 제주 제주시 #제주숙소 #제주감성숙소 #제주숙소추천 남해군 서면 춘천시 남해 광안리 편 반 안면도 산방산 님 사장 정말 날 꼭 찾다 직접 날 그냥 맘 근데 스타 욕 편 째 첫 맛 채 황리단길 미리 열다 거제도 공간 광안리 광안대교 스다 해운대 헤이 윤슬 울릉도 손 옆' #불용어 리스트 형성

In [7]:
stop_words=stop_words.split(' ')

In [8]:
stop_location = pd.read_csv('location_words.csv')
stop_location.columns = ['index', 'location']
stop_location = stop_location.location.tolist()

In [9]:
stop = stop_words + stop_location
stop = pd.Series(stop)
stopwords = stop.unique().tolist()

In [10]:
kiwi = Kiwi()
kiwi.prepare()
# with open ('stopwords_korean.txt','r', encoding = 'utf8') as f:
#   stopwords =f.readline()
# stopword = stopwords.split()
# tokenize 함수를 정의합니다. 한국어 문장을 입력하면 형태소 단위로 분리하고, 
# 불용어 및 특수 문자 등을 제거한 뒤, list로 반환합니다.

def tokenize(sent):
    res, score = kiwi.analyze(sent)[0] # 첫번째 결과를 사용
    return [word + ('다' if tag.startswith('V') else '') # 동사에는 '다'를 붙여줌
            for word, tag, _, _ in res
            if not tag.startswith('E') and not tag.startswith('J') and not tag.startswith('S') and not tag.startswith('W') and word not in stopwords] # 조사, 어미, 특수기호 및 stopwords에 포함된 단어는 제거



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: `prepare()` has no effect and will be removed in future version.
  


In [11]:
# # import time
# # ans=[]
# for i in range(len(text)):
#   text[i]= tokenize(text[i])

In [12]:
# text

In [13]:
sample = text.copy()

In [14]:
#"""한글빼고 전부 제거"""
def sub_special(s):
  rs = re.sub(r'[^가-힣]',' ',s)
  rr = re.sub(' +', ' ', rs)
  return rr

In [15]:
for i in range(len(sample)):
  sample[i] = sub_special(sample[i]) 

In [16]:
sample

0      속초 파도의무늬오션뷰 감성 에어비앤비 파도의무늬 파도의무늬 는물결이 일렁이고 윤슬이...
1      강릉 스테이 시호일시간의 흔적을 담은 감성숙소 스테이 시호일 스테이 시호일 은 년대...
2      제주 까만돌멩이구좌읍 조식주는 예쁜 숙소 까만돌멩이 까만돌멩이 는가족 및 커플 또는...
3      단양 권장사양인스타감성 가득한 독채 숙소 권장사양 권장사양 은 자연경관으로 둘러쌓여...
4      강릉 라온가따뜻한 느낌의 힐링 감성숙소 라온가 라온가 는 주택을 개조한 숙소이며 라...
                             ...                        
604     한 폭의 그림같은 오션뷰 제주 액자 창문 너머로 보이는 바다만 보고 있어도 시간 ...
605     통창 리버뷰가 환상적인 춘천 피그멜리온이펙트 리버 마운틴뷰를 보며 월풀스파를 즐길...
606    작은 시골 동네 안에 위치한 오붓 편안한 감성숙소 곡성 품안의밤 논밭뷰를 보며 여유...
607     에서 방송됐던 한 번 다녀왔습니다 의 웨딩촬영지로 유명한 강릉 더브리앙뜨 화이트 ...
608     고풍스런 미칀 한옥뷰맛집 제주 한라궁호텔 아침에 눈을 뜨면 보이는 한옥정원뷰 객실...
Name: content, Length: 609, dtype: object

In [17]:
import time

for i in range(len(sample)):
  sample[i]= tokenize(sample[i])

In [18]:
empty = []
for i in range(len(sample)): # 빈 리스트 확인
  if len(sample[i]) <= 0:
    empty.append(i)

In [19]:
text[149]

'✅ #전주_다락..전주 한옥마을의 고풍스러운 한옥에서 소중한 추억을 선물 받을수 있는 "다락" 시골 할머니집에 놀러온듯한 느낌을 주는 아늑하면서 포근한 공간이다..전통적인 한옥을 경험해볼수 있는 다락1과 모던함과 한옥을 잘 어우러지도록 만든 다락2로 구성되어있으며 2인실부터 6인실까지 다양하게 갖춰져있어서 친구, 연인, 가족들과 함께 취향대로 골라갈수 있다. 마루에 앉아 정성스레 꾸며진 정원을 바라보며 도란도란 이야기 나누기 좋은 공간..아기자기한 소품 안에서는 주인장의 감성을 담아 더욱 옛스러움을 느낄수 있는 공간. 한옥마을 안 경기전 옆에 위치해 있기때문에 주변 관광하기에도 좋다..✔ 더 많은 전주숙소는 #감성숙소_전주✔ 더 많은 전라북도숙소는 #감성숙소_전라북도..#전라북도 #전주 #다락- - - - - - - - - - - - - - - - - - - - - - - - - - - - - -📍다락📍전북 전주시 완산구 경기전길 57-6📍평일 50,000원 (별하방, 비수기기준)📍예약문의 @darak1963 - - - - - - - -

In [20]:
empty

[149]

In [21]:
for i in empty:
  sample.drop(sample.index[i], inplace=True)

In [22]:
sample = pd.DataFrame(sample)

In [23]:
sample.reset_index(inplace=True)

In [24]:
sample.drop('index', axis=1, inplace=True)

In [25]:
sample

,content
0,"[파도의무늬오션뷰, 무늬, 무늬, 물결, 일렁이다, 반짝거리다, 철썩, 철썩, 치다..."
1,"[시호일시간, 흔적, 담다, 시호, 시호, 년대, 번화가, 유명, 흔적, 품다, 주..."
2,"[까맣다, 돌멩이, 조식, 예쁘다, 까맣다, 돌멩이, 까맣다, 돌멩이, 가족, 커플..."
3,"[권장, 가득, 권장, 권장, 자연경관, 두르다, 쌓이다, 마당, 힐, 링, 판, ..."
4,"[라온가, 따뜻, 힐링, 라온, 온, 주택, 개조, 라온, 우리말, 즐겁다, 뜻, ..."
...,...
603,"[폭, 그림, 오션뷰, 액자, 창문, 너머, 바다, 감성숙소랍니돠, 미니멀리즘, 추..."
604,"[리버뷰, 피그멜리온이펙, 트다, 리버, 마운틴뷰, 월풀스파, 즐기다, 뷰맛, 원룸..."
605,"[시골, 동네, 오붓, 편안, 밤, 논밭뷰, 여유, 만끽, 힐링, 게스트하우스, 사..."
606,"[다녀오다, 웨딩촬영지, 유명, 더브리앙, 뜨다, 화이트, 앤, 우드, 톤, 좋아하..."


In [26]:
stop_words = '의 가 이 은 들 는 좀 잘 속초 걍 과 했 거 해서 ᆷ 게 찍 느낌 많이 실시간 퇴 인생 숙박 홈 즈 해변 트다 ㅂ 권 듯 뷰 박 링크 인스타그램 할인 성수기 홈페이지 주말 블로그 인원 추가 ㅎㅎ 너무 게스트 넘 하우스 드리다 이용 위치 쓰다 진짜 넘 찍 거 먹 ㅠㅠ ㅎㅎㅎ ㅠ 물 였 ㅠㅠㅠ ㅠㅠ ㅠ ㅋㅋㅋ ㅋㅋ ㅋ ㅎㅎㅎ ㅎㅎ ㅎ 화장실 도 뭐 오픈 최대 준비 룸 빵 거 많이 방법 달리 스럽다 특별자치도 를 에어비 으로 자 에 와 한 하다 아 휴 아이구 포스팅 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 이제 분 도움 여 ㅁ ㅎ ㄶ 얼마간 둥 오랜만 약간 체크 체크아웃 가격 정보 비 수기 평일 기준 약 전국  예약 되어다 스마트 빔 블루투스 스피커 이 외 드라이어 다리미 구비 되어다 있다 주방 음식 조리 가능하다 환기 제한 전남 제부도 있다 냄새 나 요리 삼가다 경주시 부탁드리다 후 번길 문의 다 가격 수기 비성수기 주소  좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 포항 우도 양양 전주시 통영 제천 여수시 순천 고성 합천 한림읍 전주 경북 구좌읍 돌산읍 태안 하동 포항시 제주시 밀양 양평 울산 무무 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 춘천 층 남해 스테이 서울 여수 거제 추천 경주 곳 객실 이다 강원도 강원 홍천 부산 영도 호 객실 교동 풀빌라 빌라 풀 스튜디오 펜션 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 감성 숙소 호텔 제주 제주도 강원도 강릉 속초 속초시 에어비앤비 명 생각 시간 그녀 수 약 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓 사실 이렇 점 싶 말 정도 좀 원 잘 통하 ㅆ 재 채 독 롭 도란도 팅 감성비앤비 공간 참여 이벤트 당첨자 기간 부산 #부산숙소 #부산에어비앤비 #부산여행 경기 경주시 있는 한 #경주숙소 강원도 제주 제주시 #제주숙소 #제주감성숙소 #제주숙소추천 남해군 서면 춘천시 남해 광안리 편 반 안면도 산방산 님 사장 정말 날 꼭 찾다 직접 날 그냥 맘 근데 스타 욕 편 째 첫 맛 채 황리단길 미리 열다 거제도 공간 광안리 광안대교 스다 해운대 헤이 윤슬 울릉도 손 옆' #불용어 리스트 형성

In [27]:
stop_words=stop_words.split(' ')

In [28]:
stop_location = pd.read_csv('location_words.csv')
stop_location.columns = ['index', 'location']
stop_location = stop_location.location.tolist()

In [29]:
stop = stop_words + stop_location
stop = pd.Series(stop)
stopwords = stop.unique().tolist()

In [30]:
for i in range(len(sample)): #불용어 제거
  result = []
  for w in sample.content[i]:
    if w not in stopwords:
      result.append(w)
    sample.content[i] = result

In [31]:
model = tp.LDAModel(k=10, alpha=0.1, eta=0.01, min_cf=5)

for row in range(len(sample)) :
    model.add_doc(sample.content[row])

In [32]:
# model의 num_words나 num_vocabs 등은 train을 시작해야 확정됩니다.
# 따라서 이 값을 확인하기 위해서 train(0)을 하여 실제 train은 하지 않고
# 학습 준비만 시킵니다.
# num_words, num_vocabs에 관심 없다면 이부분은 생략해도 됩니다.
model.train(0) 
print('Total docs:', len(model.docs))
print('Total words:', model.num_words)
print('Vocab size:', model.num_vocabs)

Total docs: 608
Total words: 49958
Vocab size: 1543


In [ ]:
# 다음 구문은 train을 총 200회 반복하면서, 
# 매 단계별로 로그 가능도 값을 출력해줍니다.
# 혹은 단순히 model.train(200)으로 200회 반복도 가능합니다.
for i in range(200):
    print('Iteration {}\tLL per word: {}'.format(i, model.ll_per_word))
    model.train(1)

In [34]:
# 학습된 토픽들을 출력해보도록 합시다.
for i in range(model.k):
    # 토픽 개수가 총 20개이니, 0~19번까지의 토픽별 상위 단어 10개를 뽑아봅시다.
    res = model.get_topic_words(i, top_n=10)
    print('Topic #{}'.format(i), end='\t')
    print(', '.join(w for w, p in res))

Topic #0	다녀오다, 이쁘다, 활용, 보내다, 상품권, 여행, 제작, 태그해, 콘탠츠, ᆸ
Topic #1	경기도, 캠핑, 여행, 근교, 핑, 산, 카라, 유럽, 호수, 글램핑
Topic #2	여행, 꾸미다, 소리, 비수기, 바라보다, 경상북도, 풍경, 햇살, 즐기다, 힐링
Topic #3	하루, 마을, 민박, 즐기다, 힐링, 독채, 마당, 숲, 휴식, 사용
Topic #4	여행, 국내, 기록, 여행지, 핫, 예쁘다, 또는, 프다, 만원, 커플
Topic #5	즐기다, 바베큐, 여행, 제공, 가족, 노을, 야외, 프라이, 프라이빗, 다양
Topic #6	조식, 여행, 충남, 여행지, 충청남도, 제공, 마운틴뷰, 카페, 한적, 에디터
Topic #7	한옥, 전라북도, 고즈넉, 느끼다, 현대, 여행, 한옥스테, 전통, 매력, 책
Topic #8	바다, 수영장, 오션뷰, 즐기다, 경남, 바라보다, 경상남도, 여행, 탕, 리조트
Topic #9	비애, 느끼다, 떠나다, 따뜻, 출처, 계획, 호캉, 인테리어, 가득, 예쁘다


In [ ]:
topic_term_dists = np.stack([model.get_topic_word_dist(k) for k in range(model.k)])
doc_topic_dists = np.stack([doc.get_topic_dist() for doc in model.docs])
doc_topic_dists /= doc_topic_dists.sum(axis=1, keepdims=True)
doc_lengths = np.array([len(doc.words) for doc in model.docs])
vocab = list(model.used_vocabs)
term_frequency = model.used_vocab_freq

prepared_data = pyLDAvis.prepare(
    topic_term_dists, 
    doc_topic_dists, 
    doc_lengths, 
    vocab, 
    term_frequency,
    start_index=0, # tomotopy starts topic ids with 0, pyLDAvis with 1
    sort_topics=False # IMPORTANT: otherwise the topic_ids between pyLDAvis and tomotopy are not matching!
)
pyLDAvis.display(prepared_data)

In [36]:
# 먼저 토픽 개수만큼 공간을 만들어줍니다.
buckets = [[] for _ in range(model.k)]
# 전체 문헌에 대해, top 1 주제 번호를 찾아 그 그룹에 해당 문헌을 포함시킵니다.
for i, d in enumerate(model.docs):
    buckets[d.get_topics(top_n=1)[0][0]].append([i,d])

In [42]:
model.docs[0]

<tomotopy.Document with words="파도의무늬오션뷰 무늬 무늬 물결 일렁이다 반짝거리다 철썩 철썩 치다 다양 모습 무늬 즐기다 들려오다 시원 소리 가만히 눈 듣다 계획 느긋 머리 식히다 세 엿보이다 푸르다 무늬 오후 저녁 들어오다 따뜻 영랑호 햇살 그대로 담다 바다 오션뷰에 욕실 분리 눈 소리 듣다 티 스트레스 티 복잡 마음 해소 무늬 정확 표시 비수기 데일리 떠나다 데일리 여행 데일리">

In [58]:
topic_table = pd.DataFrame()
doc_index = []
content = []
topics = []
for i in range(len(model.docs)):
  doc_index = int(i)
  content = sample.content[i]
  topics = int(model.docs[i].get_topics(top_n=1)[0][0])
  topic_table = topic_table.append(pd.Series([doc_index, content, topics]), ignore_index=True)

In [60]:
topic_table.columns = ['index', 'content', 'topic']

In [61]:
topic_table

,index,content,topic
0,0.0,"[파도의무늬오션뷰, 무늬, 무늬, 물결, 일렁이다, 반짝거리다, 철썩, 철썩, 치다...",7.0
1,1.0,"[시호일시간, 흔적, 담다, 시호, 시호, 년대, 번화가, 유명, 흔적, 품다, 주...",9.0
2,2.0,"[까맣다, 돌멩이, 조식, 예쁘다, 까맣다, 돌멩이, 까맣다, 돌멩이, 가족, 커플...",9.0
3,3.0,"[권장, 가득, 권장, 권장, 자연경관, 두르다, 쌓이다, 마당, 힐, 링, 판, ...",2.0
4,4.0,"[라온가, 따뜻, 힐링, 라온, 온, 주택, 개조, 라온, 우리말, 즐겁다, 뜻, ...",9.0
...,...,...,...
603,603.0,"[폭, 그림, 오션뷰, 액자, 창문, 너머, 바다, 감성숙소랍니돠, 미니멀리즘, 추...",0.0
604,604.0,"[리버뷰, 피그멜리온이펙, 리버, 마운틴뷰, 월풀스파, 즐기다, 뷰맛, 원룸, 형,...",0.0
605,605.0,"[시골, 동네, 오붓, 편안, 밤, 논밭뷰, 여유, 만끽, 힐링, 게스트하우스, 사...",3.0
606,606.0,"[다녀오다, 웨딩촬영지, 유명, 더브리앙, 뜨다, 화이트, 앤, 우드, 톤, 좋아하...",0.0


In [47]:
topic_table = make_topictable_per_doc(model, sample)

In [49]:
print(topic_table)

None


In [ ]:
def make_topictable_per_doc(ldamodel, corpus):
    topic_table = pd.DataFrame()

    # 몇 번째 문서인지를 의미하는 문서 번호와 해당 문서의 토픽 비중을 한 줄씩 꺼내온다.
    for i, topic_list in enumerate(ldamodel[corpus]):
        doc = topic_list[0] if ldamodel.get_topics else topic_list            
        doc = sorted(doc, key=lambda x: (x[1]), reverse=True)
        # 각 문서에 대해서 비중이 높은 토픽순으로 토픽을 정렬한다.
        # EX) 정렬 전 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (10번 토픽, 5%), (12번 토픽, 21.5%), 
        # Ex) 정렬 후 0번 문서 : (2번 토픽, 48.5%), (8번 토픽, 25%), (12번 토픽, 21.5%), (10번 토픽, 5%)
        # 48 > 25 > 21 > 5 순으로 정렬이 된 것.

        # 모든 문서에 대해서 각각 아래를 수행
        for j, (topic_num, prop_topic) in enumerate(doc): #  몇 번 토픽인지와 비중을 나눠서 저장한다.
            if j == 0:  # 정렬을 한 상태이므로 가장 앞에 있는 것이 가장 비중이 높은 토픽
                topic_table = topic_table.append(pd.Series([int(topic_num), round(prop_topic,4), topic_list]), ignore_index=True)
                # 가장 비중이 높은 토픽과, 가장 비중이 높은 토픽의 비중과, 전체 토픽의 비중을 저장한다.
            else:
                break
    return(topic_table)

In [ ]:
import tomotopy as tp

corpus = tp.utils.Corpus(tokenizer=tp.utils.SimpleTokenizer(), stopwords=['.'])
# data_feeder yields a tuple of (raw string, user data) or a str (raw string)
corpus.process(text)

# make LDA model and train
mdl = tp.LDAModel(k=20, min_cf=10, min_df=5, corpus=corpus)
mdl.train(0)
print('Num docs:', len(mdl.docs), ', Vocab size:', len(mdl.used_vocabs), ', Num words:', mdl.num_words)
print('Removed top words:', mdl.removed_top_words)
for i in range(0, 1000, 10):
    mdl.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

# extract candidates for auto topic labeling
extractor = tp.label.PMIExtractor(min_cf=10, min_df=5, max_len=5, max_cand=10000)
cands = extractor.extract(mdl)

# ranking the candidates of labels for a specific topic
labeler = tp.label.FoRelevance(mdl, cands, min_df=5, smoothing=1e-2, mu=0.25)
for k in range(mdl.k):
    print("== Topic #{} ==".format(k))
    print("Labels:", ', '.join(label for label, score in labeler.get_topic_labels(k, top_n=5)))
    for word, prob in mdl.get_topic_words(k, top_n=10):
        print(word, prob, sep='\t')
    print()


##Konlpy

In [ ]:
hannanum = Hannanum()
okt = Okt()

In [ ]:
data = data[['name','content']] #데이터셋 재구성

In [ ]:
#"""한글빼고 전부 제거"""
# def sub_special(s):
#   return re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣]','',s)

In [ ]:
# sub_special(data['content'])

In [ ]:
stop_words=stop_words.split(' ')

In [ ]:
text_okt = data.content[0]

In [ ]:
text_okt

In [ ]:
def tokenize(text) : #토큰화 명사 동사 형용사만 추출
    result = []
    word_s = okt.pos(text, norm=True, stem=True)
    for n, h in word_s :
        if not (h in ['Noun', 'Verb', 'Adjective']) : continue
        result.append(n)
    return result

In [ ]:
token_okt = tokenize(text_okt)

In [ ]:
result = [] #불용어제거
for w in token_okt: 
    if w not in stop_words: 
        result.append(w)

# print(text_okt,'\n')
print(result)

In [ ]:
data['token_content'] = np.NaN

In [ ]:
for i in range(len(data.content)):
  data.content[i] = tokenize(data.content[i])

In [ ]:
for i in range(len(data.content)):
  result = []
  for w in data.content[i]:
    if w not in stop_words:
      result.append(w)
    data.token_content[i] = result

In [ ]:
data

In [ ]:
data['token_nolist'] = np.NaN

In [ ]:
for i in range(len(data)):
  data.token_nolist[i] = " ".join(data.token_content[i])

In [ ]:
" ".join(data.token_content[0])

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
test = data.token_nolist

In [ ]:
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(test)
# 줄거리에 대해서 tf-idf 수행
print(tfidf_matrix.shape)

Kmeans 클러스터링

In [ ]:
from gensim.models import Word2Vec, FastText

# 64차원으로 임베딩
cbow = Word2Vec(data.content, min_count=1, size=64)
# fasttext = FastText(data.content, min_count=1, size=64)

# cbow를 선택한것은 리뷰는 집약적으로 표현한 데이터이므로
# 전체적인 통계정보나 subword의 의미를 파악하기보다는 각 단어의 유사도를 판단하는 것이 중요하다고 생각했습니다.

In [ ]:
print(cbow.wv['월림차경'])

In [ ]:
print(cbow.wv.most_similar('월림차경'))

In [ ]:
print (len(cbow.wv.vocab))

In [ ]:
X = cbow[cbow.wv.vocab]
# 클러스터링하기위해 X에다 데이터 넣기
# 임베딩된 벡터들이 들어가게 된다

In [ ]:
from sklearn.cluster import KMeans
from sklearn.neighbors import KDTree
import numpy as np

# 단어벡터들을 n개의 클러스터로 군집화 해주는 함수 kmeans++을 사용하였다.
def clustering_on_wordvecs(word_vectors, num_clusters):
    # Initalize a k-means object and use it to extract centroids
    kmeans_clustering = KMeans(n_clusters = num_clusters, init='k-means++')
    idx = kmeans_clustering.fit_predict(word_vectors)
    
    return kmeans_clustering.cluster_centers_, idx

# 출처 : https://medium.com/ml2vec/using-word2vec-to-analyze-reddit-comments-28945d8cee57
# 대부분 그대로 사용하였고, 파리미터의 수를 조정하여 사용하였다.

In [ ]:
centers, clusters = clustering_on_wordvecs(X, 3)
centroid_map = dict(zip(cbow.wv.index2word, clusters))

In [ ]:
pd.Series(clusters).value_counts() # 압도적으로 1개 주제가 제일 많음을 볼 수 있다. 

In [ ]:
# 각 클러스터에 중심점에서 가장 유사한 벡터들(단어들) k개를 뽑는 함수
def get_top_words(index2word, k, centers, wordvecs):
    tree = KDTree(wordvecs);
    #Closest points for each Cluster center is used to query the closest 20 points to it.
    closest_points = [tree.query(np.reshape(x, (1, -1)), k=k) for x in centers];
    closest_words_idxs = [x[1] for x in closest_points];
    #Word Index is queried for each position in the above array, and added to a Dictionary.
    closest_words = {};
    for i in range(0, len(closest_words_idxs)):
        closest_words['Cluster #' + str(i)] = [index2word[j] for j in closest_words_idxs[i][0]]
    #A DataFrame is generated from the dictionary.
    df = pd.DataFrame(closest_words);
    df.index = df.index+1
    return df

In [ ]:
top_words = get_top_words(cbow.wv.index2word, 50, centers, X)

In [ ]:
top_words.head()

시각화

In [ ]:
import matplotlib as mat
import matplotlib.pyplot as plt

In [ ]:
# 군집화된 단어들을 워드클라우드로 그려서 보여주는 함수
def display_cloud(cluster_num, cmap):
    wc = WordCloud(
        font_path = "/content/NanumGothic.ttf",
        width = 2000,
        height = 1500,
        background_color="black",
        max_words=20,
        colormap=cmap)
    wordcloud = wc.generate(' '.join([word for word in top_words['Cluster #' + str(cluster_num-1).zfill(1)]]))
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.savefig('이전cluster_' + str(cluster_num), bbox_inches='tight')

In [ ]:
from itertools import cycle
from wordcloud import WordCloud, ImageColorGenerator

cmaps = cycle([
            'flag', 'prism', 'ocean', 'gist_earth', 'terrain', 'gist_stern',
            'gnuplot', 'gnuplot2', 'CMRmap', 'cubehelix', 'brg', 'hsv',
            'gist_rainbow', 'rainbow', 'jet', 'nipy_spectral', 'gist_ncar'])

centers, clusters = clustering_on_wordvecs(X, 3);
centroid_map = dict(zip(cbow.wv.index2word, clusters));
top_words = get_top_words(cbow.wv.index2word, 50, centers, X)


In [ ]:
for i in range(3):
    col = next(cmaps);
    display_cloud(i+1,col)

doc2vec

In [ ]:
!pip install tomotopy

In [ ]:
import tomotopy as tp
print(tp.isa) # 'avx2'나 'avx', 'sse2', 'none'를 출력합니다.

In [ ]:
import tomotopy as tp
mdl = tp.LDAModel(k=20)
for line in file:
    mdl.add_doc(line.strip().split())

for i in range(0, 100, 10):
    mdl.train(10)
    print('Iteration: {}\tLog-likelihood: {}'.format(i, mdl.ll_per_word))

for k in range(mdl.k):
    print('Top 10 words of topic #{}'.format(k))
    print(mdl.get_topic_words(k, top_n=10))

mdl.summary()

## 유사도 분석

In [ ]:
from sklearn.metrics.pairwise import linear_kernel
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)

In [ ]:
cosine_sim

In [ ]:
##인덱스 테이블 만들기##
indices = pd.Series(data.index, index=data.name).drop_duplicates()
print(indices)

In [ ]:
def insta_REC(name, cosine_sim=cosine_sim):
    #입력한 숙소로 부터 인덱스 가져오기
    idx = indices[name]

    # 모든 숙소에 대해서 해당 숙소와의 유사도를 구하기
    sim_scores = list(enumerate(cosine_sim[idx]))

    # 유사도에 따라 숙소를 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 숙소를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 숙소의 인덱스 받아옴
    insta_indices = [i[0] for i in sim_scores]
    
    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_data = data.iloc[insta_indices].copy()
    result_data['score'] = [i[1] for i in sim_scores]
    
    # 읽어들인 데이터에서 줄거리 부분만 제거, 제목과 스코어만 보이게 함
    del result_data['content']

    # 가장 유사한 10개의 숙소의 제목을 리턴
    return result_data
    

In [ ]:
insta_REC('월림차경')

Adjusted cosine_Similarity

평균값을 고려하여 유사도 측정

In [ ]:
tfidf_matrix

In [ ]:
!pip install utils

In [ ]:
!pip install sklearn
!pip install joblib

In [ ]:
import scipy.spatial.distance as ssd
from scipy.stats import spearmanr as spearman
from scipy.sparse import issparse
from scipy.sparse import csr_matrix

from sklearn.utils import*

In [ ]:
from sklearn.utils import safe_asarray, atleast2d_or_csr
from utils.extmath import safe_sparse_dot

In [ ]:

def check_pairwise_arrays(X, Y):
    """ Set X and Y appropriately and checks inputs
    If Y is None, it is set as a pointer to X (i.e. not a copy).
    If Y is given, this does not happen.
    All distance metrics should use this function first to assert that the
    given parameters are correct and safe to use.
    Specifically, this function first ensures that both X and Y are arrays,
    then checks that they are at least two dimensional. Finally, the function
    checks that the size of the second dimension of the two arrays is equal.
    Parameters
    ----------
    X : {array-like, sparse matrix}, shape = [n_samples_a, n_features]
    Y : {array-like, sparse matrix}, shape = [n_samples_b, n_features]
    Returns
    -------
    safe_X : {array-like, sparse matrix}, shape = [n_samples_a, n_features]
        An array equal to X, guaranteed to be a numpy array.
    safe_Y : {array-like, sparse matrix}, shape = [n_samples_b, n_features]
        An array equal to Y if Y was not None, guaranteed to be a numpy array.
        If Y was None, safe_Y will be a pointer to X.
    """
    if Y is X or Y is None:
        X = safe_asarray(X)
        X = Y = atleast2d_or_csr(X, dtype=np.float)
    else:
        X = safe_asarray(X)
        Y = safe_asarray(Y)
        X = atleast2d_or_csr(X, dtype=np.float)
        Y = atleast2d_or_csr(Y, dtype=np.float)
    if len(X.shape) < 2:
        raise ValueError("X is required to be at least two dimensional.")
    if len(Y.shape) < 2:
        raise ValueError("Y is required to be at least two dimensional.")
    if X.shape[1] != Y.shape[1]:
        raise ValueError("Incompatible dimension for X and Y matrices: "
                         "X.shape[1] == %d while Y.shape[1] == %d" % (
                             X.shape[1], Y.shape[1]))
    return X, Y

In [ ]:
def adjusted_cosine(X, Y, N):
    """
    Considering the rows of X (and Y=X) as vectors, compute the
    distance matrix between each pair of vectors after normalize or adjust
    the vector using the N vector. N vector contains the mean of the values
    of each feature vector from X and Y.
    This correlation implementation is equivalent to the cosine similarity
    since the data it receives is assumed to be centered -- mean is 0. The
    correlation may be interpreted as the cosine of the angle between the two
    vectors defined by the users' preference values.
    Parameters
    ----------
    X : {array-like, sparse matrix}, shape = [n_samples_1, n_features]
    Y : {array-like, sparse matrix}, shape = [n_samples_2, n_features]
    N: {array-like, sparse matrix}, shape = [n_samples_3, n_features]
    Returns
    -------
    distances : {array, sparse matrix}, shape = [n_samples_1, n_samples_2]
    Examples
    --------
    >>> from crab.metrics.pairwise import adjusted_cosine
    >>> X = [[1.0, 5.0, 4.0]]
    >>> Y = [[2.0, 5.0, 5.0]]
    >>> N = [[3.0, 3.5, 4.0]]
    >>> # distance between rows of X
    >>> adjusted_cosine(X, X, N)
    array([[ 1.]])
    >>> adjusted_cosine(X, Y, N)
    array([[ 0.82462113]])
    """

    X, Y = check_pairwise_arrays(X, Y)
    #TODO: fix next line
    N, _ = check_pairwise_arrays(N, None)

    # should not need X_norm_squared because if you could precompute that as
    # well as Y, then you should just pre-compute the output and not even
    # call this function.

    #TODO: Fix to work with sparse matrices.
    if issparse(X) or issparse(Y) or issparse(N):
        raise ValueError('Adjusted cosine does not yet support sparse matrices.')

    if X is Y:
        X = Y = np.asanyarray(X)
    else:
        X = np.asanyarray(X)
        Y = np.asanyarray(Y)

    if X.shape[1] != Y.shape[1] != N.shape[1]:
        raise ValueError("Incompatible dimension for X, Y and N matrices")

    X = X - N
    Y = Y - N

    XY = 1 - ssd.cdist(X, Y, 'cosine')

    return XY

In [ ]:
adjusted_cosine(tfidf_matrix,tfidf_matrix,tfidf_matrix)

In [ ]:
mu_a

In [ ]:
mu_b

In [ ]:
norm_a=np.subtract(tfidf_matrix,mu_a)
norm_b=np.subtract(tfidf_matrix,mu_b)

In [ ]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity


def adjusted_cosine_similarity(a, b):
  
    a = np.array(a)
    b = np.array(b)
    
    mu_a = np.mean(a)
    mu_b = np.mean(b)
    
    norm_a = a - mu_a
    norm_b = b - mu_b
    
    return cosine_similarity(norm_a, norm_b)

In [ ]:
adjusted_cosine_similarity(tfidf_matrix, tfidf_matrix)

In [ ]:
adjusted=cosine_similarity(tfidf_matrix,tfidf_matrix)

In [ ]:
def insta_REC(name, adjusted=adjusted):
    #입력한 숙소로 부터 인덱스 가져오기
    idx = indices[name]

    # 모든 숙소에 대해서 해당 숙소와의 유사도를 구하기
    sim_scores = list(enumerate(adjusted[idx]))

    # 유사도에 따라 숙소를 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1], reverse = True)

    # 가장 유사한 10개의 숙소를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 숙소의 인덱스 받아옴
    insta_indices = [i[0] for i in sim_scores]
    
    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 코사인 유사도도 확인할 수 있게 한다.
    result_data = data.iloc[insta_indices].copy()
    result_data['score'] = [i[1] for i in sim_scores]
    
    # 읽어들인 데이터에서 줄거리 부분만 제거, 제목과 스코어만 보이게 함
    del result_data['content']

    # 가장 유사한 10개의 숙소의 제목을 리턴
    return result_data
    

In [ ]:
insta_REC('월림차경')

유클리디언 유사도 ( L2 Distance )
- 가장 기본적인 거리를 측정하는 유사도 공식이다.
- 값이 작을 수록 거리가 가까움 (제일 유사함)

In [ ]:
from sklearn.metrics.pairwise import euclidean_distances

euclidean_distances(tfidf_matrix[0:1], tfidf_matrix[1:2])

In [ ]:
import numpy as np

def l1_normalize(v):
  norm = np.sum(v)
  return v / norm
    
    
tfidf_norm_l1 = l1_normalize(tfidf_matrix) #tf-idf 값 정규화

euclidean_distances=euclidean_distances(tfidf_norm_l1)

In [ ]:
euclidean_distances

In [ ]:
def insta_REC(name, euclidean_distances = euclidean_distances):
    #입력한 숙소명으로 부터 인덱스 가져오기
    idx = indices[name]

    # 수집된 숙소에 대해서 해당 숙소와 유사한 content의 유사도를 구하기
    sim_scores = list(enumerate(euclidean_distances[idx]))

    # 유사도에 따라 숙소 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1])

    # 가장 유사한 10개의 숙소이름을 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 영화의 인덱스 받아옴
    insta_indices = [i[0] for i in sim_scores]
    
    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여 유사도도 확인할 수 있게 한다.
    result_data = data.iloc[insta_indices].copy()
    result_data['score'] = [i[1] for i in sim_scores]
    
    # 읽어들인 데이터에서 숙소명과 contents만 보이게 함
    del result_data['content']

    # 가장 유사한 10개의 숙소명을 리턴
    return result_data

In [ ]:
insta_REC('월림차경')

맨하탄 유사도

In [ ]:
# 맨하탄 유사도
from sklearn.metrics.pairwise import manhattan_distances

manhattan_distances = manhattan_distances(tfidf_norm_l1)

In [ ]:
def insta_REC(name, manhattan_distances=manhattan_distances):
    #입력한 숙소로 부터 인덱스 가져오기
    idx = indices[name]

    # 모든 숙소에 대해서 해당 숙소와의 유사도를 구하기
    sim_scores = list(enumerate(manhattan_distances[idx]))

    # 유사도에 따라 숙소들을 정렬
    sim_scores = sorted(sim_scores, key=lambda x:x[1])

    # 가장 유사한 10개의 숙소를 받아옴
    sim_scores = sim_scores[1:11]

    # 가장 유사한 10개 숙소의 인덱스 받아옴
    insta_indices = [i[0] for i in sim_scores]
    
    #기존에 읽어들인 데이터에서 해당 인덱스의 값들을 가져온다. 그리고 스코어 열을 추가하여유사도도 확인할 수 있게 한다.
    result_data = data.iloc[insta_indices].copy()
    result_data['score'] = [i[1] for i in sim_scores]
    
    # 읽어들인 데이터에서, 숙소명과 content만 보이게 함
    del result_data['content']

    # 가장 유사한 10개의 숙소를 리턴
    return result_data

In [ ]:
insta_REC('월림차경')